In [49]:
video_folder = "AlgonautsVideos268_All_30fpsmax"

In [2]:
import os
import gc
import cv2
import numpy as np
import pickle
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing import image
from keras.models import Model

In [3]:
# Initialize ResNet50 model pre-trained on ImageNet
base_model = ResNet50(weights='imagenet')

# Select layers for feature extraction at the end of each stage
layers_to_extract = {
    'stage_1': base_model.get_layer('pool1_pool').output,
    'stage_2': base_model.get_layer('conv2_block3_out').output,
    'stage_3': base_model.get_layer('conv3_block4_out').output,
    'stage_4': base_model.get_layer('conv4_block6_out').output,
    'stage_5': base_model.get_layer('conv5_block3_out').output,
    'final': base_model.get_layer('predictions').output
}

# Create models for each stage
models = {stage: Model(inputs=base_model.input, outputs=output) for stage, output in layers_to_extract.items()}



102967424/102967424 [==============================] - 4s 0us/step


In [4]:
def preprocess_video(video_path, frame_rate=15):
    # Capture video
    vidcap = cv2.VideoCapture(video_path)
    success, frame = vidcap.read()
    count = 0
    features_per_stage = {stage: [] for stage in layers_to_extract}

    while success:
        if count % (30 // frame_rate) == 0:  # Reduce frame rate
            # Resize frame to match the input size expected by ResNet50
            resized_frame = cv2.resize(frame, (224, 224))  # Resize to 224x224

            # Preprocess frame
            img = image.img_to_array(resized_frame)
            img = np.expand_dims(img, axis=0)
            img = preprocess_input(img)

            # Extract and store features for each stage
            for stage, model in models.items():
                stage_features = model.predict(img)
                features_per_stage[stage].append(stage_features)
        
        success, frame = vidcap.read()
        count += 1

    # Stack features for 3D input
    stacked_features = {stage: np.stack(features, axis=0) for stage, features in features_per_stage.items()}

    return stacked_features


In [90]:
# Example usage
video_tensors = preprocess_video('AlgonautsVideos268_All_30fpsmax/0002_0-0-4-3146384004.mp4')

1/1 [==============================] - 2s 2s/step


In [5]:
type(video_tensors)

NameError: name 'video_tensors' is not defined

In [46]:
def set_wd_to_root():
    # Name of the root directory to which we want to change the working directory
    root_directory_name = "Brainvision_Project"

    # Get the current working directory
    current_directory = os.getcwd()
    print(f"Current working directory: {current_directory}")

    # Check if the last part of the current directory is the root directory
    while os.path.basename(current_directory) != root_directory_name:
        # Move up one directory
        current_directory = os.path.dirname(current_directory)

        # Check if we have reached the root directory to avoid infinite loop
        if current_directory == os.path.dirname(current_directory):
            raise Exception(f"{root_directory_name} directory not found in the path hierarchy.")
        
        # Set the working directory to the root directory
        os.chdir(current_directory)
        print(f"Working directory changed to: {os.getcwd()}")

In [47]:
def process_batch(batch, input_folder, stage_folders):
    for filename in batch:
        video_path = os.path.join(input_folder, filename)
        try:
            processed_data = preprocess_video(video_path)
            
            if not isinstance(processed_data, dict):
                raise ValueError(f"Unexpected data format for {filename}: {type(processed_data)}")

            video_id = filename[:4]
            for stage, features in processed_data.items():
                stage_folder = os.path.join(output_folder, stage)
                if not os.path.exists(stage_folder):
                    os.makedirs(stage_folder)
                    print(f"Created folder: {stage_folder}")

                output_file = os.path.join(stage_folder, f"{video_id}_{stage}.pkl")
                with open(output_file, 'wb') as file:
                    pickle.dump(features, file)
                    print(f"Saved file: {output_file}")

        except Exception as e:
            print(f"Error processing {filename}: {e}, data type: {type(processed_data)}")
            
        finally:
            # Explicitly delete large variables
            del processed_data
            # Call garbage collector
            gc.collect()

def process_and_save_videos(input_folder, output_folder, batch_size=90, id_range=None):
    # Set the working directory to "Brainvision_Project". You need to adapt the root directory in the function when working in UCloud.
    set_wd_to_root()
    
    # Ensure output folder exists
    if not os.path.exists(output_folder):
        print(f"Output does not exist yet. Create output folder {output_folder}...")
        os.makedirs(output_folder)

    # Get all MP4 files
    all_files = [f for f in os.listdir(input_folder) if f.endswith(".mp4")]

    # Filter files based on ID range if provided
    if id_range is not None:
        start_id, end_id = id_range
        all_files = [f for f in all_files if start_id <= f[:4] <= end_id]

    # Process in batches
    for i in range(0, len(all_files), batch_size):
        batch = all_files[i:i + batch_size]
        process_batch(batch, input_folder, output_folder)
        print(f"Processed batch {i // batch_size + 1}/{(len(all_files) - 1) // batch_size + 1}")


In [50]:
input_folder = video_folder  # Update with your input folder path
output_folder = 'preprocessed_videos'  # Update with your output folder path
batch_size = 90   # Adjust based on your system's capability
id_range = ('0001', '0005')  # Process videos with IDs from 1000 to 2000
process_and_save_videos(input_folder, output_folder, batch_size, id_range)

Current working directory: C:\Users\marce\PycharmProjects\Brainvision_Project
1/1 [==============================] - 0s 78ms/step
Created folder: preprocessed_videos\stage_1
Saved file: preprocessed_videos\stage_1\0001_stage_1.pkl
Created folder: preprocessed_videos\stage_2
Saved file: preprocessed_videos\stage_2\0001_stage_2.pkl
Created folder: preprocessed_videos\stage_3
Saved file: preprocessed_videos\stage_3\0001_stage_3.pkl
Created folder: preprocessed_videos\stage_4
Saved file: preprocessed_videos\stage_4\0001_stage_4.pkl
Created folder: preprocessed_videos\stage_5
Saved file: preprocessed_videos\stage_5\0001_stage_5.pkl
Created folder: preprocessed_videos\final
Saved file: preprocessed_videos\final\0001_final.pkl
1/1 [==============================] - 0s 80ms/step
Saved file: preprocessed_videos\stage_1\0002_stage_1.pkl
Saved file: preprocessed_videos\stage_2\0002_stage_2.pkl
Saved file: preprocessed_videos\stage_3\0002_stage_3.pkl
Saved file: preprocessed_videos\stage_4\0002_st

In [ ]:
import cv2
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image
import numpy as np

# Load the ResNet50 model pre-trained on ImageNet data
model = ResNet50(weights='imagenet')

def classify_mid_frame(video_path):
    # Capture the video
    vidcap = cv2.VideoCapture(video_path)

    # Get the frame in the middle of the video
    total_frames = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
    mid_frame_index = int(total_frames / 2)
    vidcap.set(cv2.CAP_PROP_POS_FRAMES, mid_frame_index)
    success, mid_frame = vidcap.read()
    
    if not success:
        return "Could not read the video file"

    # Convert the frame to a format suitable for ResNet50
    img = cv2.cvtColor(mid_frame, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    # Classify the image
    preds = model.predict(x)
    return decode_predictions(preds, top=3)[0]


In [ ]:
# Example usage
video_path = 'AlgonautsVideos268_All_30fpsmax/0006_0-1-9-9-8-6-5-9-4701998659.mp4'
predictions = classify_mid_frame(video_patxh)
print(predictions)